In [1]:
using HTTP, JSON, PrettyTables, JLD, MySQL, DotEnv, DataFrames, LinearAlgebra, Dates
include("functions/type_allocation_flexible.jl")
cfg = DotEnv.config("../.env")
#if you want to refresh the adjustment information set refresh to true
# to load saved data instead (which means you'll just be reading previously saved 
# adjustments, set refresh to false
refresh = true
files_path = cfg["files_path"]

"/home/peters/code/mapinator/estimation/current_estimates_and_files/"

In [2]:
adjusted_placement_rates = load(files_path*"adjusted_placement_rates.jld")["adjusted_placement_rates"]

12×5 Matrix{Int64}:
 1424   469   326   110   11
 1133  1184   595   232   28
 1280  1917  1869   373   96
  322   703   855  1044   93
    0    58   100    71  459
  678   757   540   145   65
  927   612   436   140   73
  103   180   256   143   76
   41    82   131   105  135
  438   832  1426  1054  959
  251   229   157    82   25
  286   523   796   406  194

In [3]:
 A = adjusted_placement_rates./sum(adjusted_placement_rates,dims = 1)

12×5 Matrix{Float64}:
 0.206887   0.0621521   0.0435421  0.028169   0.00496838
 0.164608   0.156904    0.0794711  0.059411   0.0126468
 0.185965   0.254042    0.249633   0.0955186  0.0433604
 0.0467819  0.0931619   0.114198   0.26735    0.0420054
 0.0        0.00768619  0.0133565  0.0181818  0.207317
 0.0985036  0.100318    0.072125   0.0371319  0.0293586
 0.13468    0.0811026   0.0582343  0.0358515  0.032972
 0.0149644  0.0238537   0.0341926  0.0366197  0.034327
 0.0059567  0.0108667   0.017497   0.0268886  0.0609756
 0.063635   0.110257    0.190463   0.26991    0.433153
 0.0364667  0.0303472   0.0209697  0.0209987  0.0112918
 0.0415516  0.0693082   0.106318   0.103969   0.0876242

In [4]:
classification_properties = load(files_path*"classification_properties.jld")["classification_properties"]

Dict{String, Any} with 7 entries:
  "data_loaded"              => DateTime("2024-06-18T18:21:53.103")
  "row_labels"               => Any["TYPE 1 (20 insts)", "TYPE 2 (58 insts)", "…
  "unmatched_index"          => 10
  "number_of_academic_types" => 5
  "institution_counts"       => [20, 58, 180, 334, 522, 152, 227, 598, 413, 1, …
  "algorithm_run_id"         => 5
  "num_years"                => 24

In [5]:
expected_offers = load(files_path*"expected_offers.jld")["expected_offers"]

11-element Vector{Any}:
 0.6070839143953419
 0.48690836585935493
 0.40754303886040527
 0.1635911209966943
 0.006460048111407303
 0.43583456388151104
 0.47712510196411917
 0.24272356619594543
 0.1279962497856872
 0.41923816133868286
 0.2434478627925604

In [6]:

offer_values = []
for j in 1:length(expected_offers)
    if j < classification_properties["unmatched_index"]-1
        push!(offer_values, expected_offers[j])
    elseif j == classification_properties["unmatched_index"]-1
        push!(offer_values, expected_offers[j])
        push!(offer_values, 0.0)
    else
        push!(offer_values,expected_offers[j-1])
    end
end


In [7]:
transpose(offer_values)*A

1×5 transpose(::Vector{Any}) with eltype Any:
 0.422861  0.355493  0.302648  0.220705  0.117901

In [8]:
placement_rates = load(files_path*"placement_rates.jld")["placement_rates"]

12×5 Matrix{Int32}:
 1194   338  172   43    4
  950   853  314   91   10
 1073  1381  986  146   34
  270   506  451  409   33
    0    42   53   28  163
  568   545  285   57   23
  777   441  230   55   26
   86   130  135   56   27
   34    59   69   41   48
  367   599  752  413  341
  210   165   83   32    9
  240   377  420  159   69

In [10]:
row_names = load(files_path*"row_names.jld")["names"]

12-element Vector{Any}:
 "TYPE 1 (20 insts)"
 "TYPE 2 (58 insts)"
 "TYPE 3 (180 insts)"
 "TYPE 4 (334 insts)"
 "TYPE 5 (522 insts)"
 "Public Sector (152 insts)"
 "Private Sector (227 insts)"
 "Postdocs (598 insts)"
 "Lecturers (413 insts)"
 "Unmatched (1 insts)"
 "Other Groups (38 insts)"
 "Teaching Universities (642 insts)"

In [16]:
column_sums = sum(placement_rates, dims=1)
    row_sums = sum(placement_rates, dims=2)
    row_sums_augmented = vcat(row_sums, sum(row_sums))
    part = vcat(placement_rates,column_sums)
    adjacency = hcat(part, row_sums_augmented)
    num = size(placement_rates, 2)
    headers = []
    for i=1:num
        push!(headers, "Tier $i")
    end
    
    #println(headers)

    push!(headers, "Row Totals")
    push!(row_names, "Column Totals")
pretty_table(adjacency, header= headers, row_labels=row_names, backend = Val(:markdown))

|                                   | **Tier 1** | **Tier 2** | **Tier 3** | **Tier 4** | **Tier 5** | **Row Totals** |
|----------------------------------:|-----------:|-----------:|-----------:|-----------:|-----------:|---------------:|
| TYPE 1 (20 insts)                 | 1194       | 338        | 172        | 43         | 4          | 1751           |
| TYPE 2 (58 insts)                 | 950        | 853        | 314        | 91         | 10         | 2218           |
| TYPE 3 (180 insts)                | 1073       | 1381       | 986        | 146        | 34         | 3620           |
| TYPE 4 (334 insts)                | 270        | 506        | 451        | 409        | 33         | 1669           |
| TYPE 5 (522 insts)                | 0          | 42         | 53         | 28         | 163        | 286            |
| Public Sector (152 insts)         | 568        | 545        | 285        | 57         | 23         | 1478           |
| Private Sector (227 insts)        | 77